In [10]:
# Preconditions

import os
import json
from PIL import Image
import numpy as np
import keras.api._v2.keras as keras
from keras.applications import VGG16

import random
import helper as hp

MODEL_NAME = "v5_data_10000" + ".keras"
MODEL_PATH = os.path.join("..", "tmp", "models")

RESSOURCES_PATH = os.path.join("..", "tmp", "train", "ressources")
TRAIN_DATA_FOLDER = "ea0205e6-c5c6-4b1d-afb6-6f0e36e7acee"

# Image properties
IMAGE_HEIGHT_PX = 120
IMAGE_WIDTH_PX = 160
CROP_HEIGHT_PX = 5
CROP_WIDTH_PX = 20

NUM_CHANNELS = 3
NUM_CLASSES = 4
NUM_POSITIONS = 8

color_mapping = { 'red': 0, 'yellow': 1, 'blue': 2, '': 3 }
label_mapping = { 0: 'red', 1: 'yellow', 2: 'blue', 3: ''}

NORMALIZE_VALUE = 255

def map_labels_to_nummeric(label):
    mapped_label = []

    for pos in label.values():
        mapped_label.append(color_mapping[pos])

    return mapped_label

# Normalize the images so that all values are between 0 and 1
def normalize_images(images):
    return images / NORMALIZE_VALUE


# Data loading
IN_DEBUG_MODE = False
IMAGE_FOLDER = "Images"
LABELS_FOLDER = "Labels"
JSON_NAME = "scene_results.json"

def get_data(stage):
    labels = []
    images = []

    scene_results_path = os.path.join(RESSOURCES_PATH, TRAIN_DATA_FOLDER, stage, JSON_NAME)

    if IN_DEBUG_MODE:
        print("CURRENT STAGE: " + stage + "\n")
        print("READING SCENE RESULTS AT: " + scene_results_path)

    with open(scene_results_path, 'r') as file:
        scene_results = json.load(file)

    for result in scene_results:
        img = [];

        image_not_found = False

        if len(result["imagePaths"]) != 2:
            continue

        for img_path in result["imagePaths"]:

            if IN_DEBUG_MODE:
                print("READING IMAGE AT: " + img_path)

            try:
                i = Image.open(os.path.join(RESSOURCES_PATH, TRAIN_DATA_FOLDER, img_path))
            except:
                image_not_found = True
                continue


            # Scale down image (resize)
            i = i.resize((IMAGE_WIDTH_PX, IMAGE_HEIGHT_PX))

            # Channel order of Pillow is different than OpenCV
            i = np.array(i)
            # i = np.array(i)[:, :, ::-1]
            i = hp.Preprocess.convert_to_BGR(i)

            i = hp.Video.translate_image(i)

            # Crop the image
            i = i[0:115, 10:150]

            i = hp.Preprocess.start(i)

            img.append(i)

        if image_not_found == False:
            images.append(img)
            try:
                # np.array(images) # is only necessary to check if the data is homogenous
                labels.append(result["positions"])
            except:
                print(f"Images shape got inhomogenous at: ${result['imagePaths']}")
                images.pop()

            

    if IN_DEBUG_MODE:
        print("\n\n")

    return [np.array(images), np.array(labels)]

In [2]:
# Base-Model generation
LOSS_FUNCTION = 'categorical_crossentropy'

input_branch_1 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))
input_branch_2 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))

# Shared convolutional layers for image processing
convolutional_layers = [
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten()
]

# Process first image
x1 = input_branch_1
for layer in convolutional_layers:
    x1 = layer(x1)

# Process second image
x2 = input_branch_2
for layer in convolutional_layers:
    x2 = layer(x2)

x = keras.layers.Concatenate(axis=-1)([x1, x2])
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(NUM_POSITIONS * NUM_CLASSES, activation='softmax')(x)  # Output layer with 8 * 4 units

output = keras.layers.Reshape((NUM_POSITIONS, NUM_CLASSES))(x)

# Build the model with the two input branches and the output layer
base_model = keras.models.Model(inputs=[input_branch_1, input_branch_2], outputs=output)

optimizer = keras.optimizers.legacy.Adam(learning_rate=0.001)

base_model.compile(optimizer=optimizer, loss=LOSS_FUNCTION, metrics=['accuracy', 'mean_squared_error'])

In [11]:

# Base model with VGG16

# Input layers for the two images
input_branch_1 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))
input_branch_2 = keras.layers.Input(shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))

# Load the pre-trained VGG16 model (without the top layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_HEIGHT_PX - CROP_HEIGHT_PX, IMAGE_WIDTH_PX - CROP_WIDTH_PX, 3))

# Freeze the pre-trained layers (optional)
for layer in base_model.layers:
  layer.trainable = False  # You can experiment with freezing some or all layers

# Extract features from both images using the VGG16 base model
x1 = base_model(input_branch_1)
x2 = base_model(input_branch_2)

# Concatenate the extracted features
x = keras.layers.Concatenate(axis=-1)([x1, x2])

# Add custom layers for your specific task
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(NUM_POSITIONS * NUM_CLASSES, activation='softmax')(x)  # Output layer with 8 * 4 units

# Final output layer for multi-class classification (adjust based on your problem)
output = keras.layers.Reshape((NUM_POSITIONS, NUM_CLASSES))(x)

# Create the final model with two inputs and one output
base_model = keras.models.Model(inputs=[input_branch_1, input_branch_2], outputs=output)

# Compile the model (adjust optimizer and loss function as needed)
base_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
# Re-train existing model
base_model = keras.models.load_model(os.path.join(MODEL_PATH, MODEL_NAME))

In [13]:
# Train model

IN_DEBUG_MODE = False
EPOCS = 10

def fit_model(model, train_data, verify_data):
    train_images = normalize_images(np.array(train_data[0]))
    numberic_train_labels = np.array([map_labels_to_nummeric(label) for label in train_data[1]])
    train_labels = keras.utils.to_categorical(numberic_train_labels, num_classes=NUM_CLASSES)

    verify_images = normalize_images(np.array((verify_data[0])))
    numberic_verify_labels = np.array([map_labels_to_nummeric(label) for label in verify_data[1]])
    verify_labels = keras.utils.to_categorical(numberic_verify_labels, num_classes=NUM_CLASSES)

    if IN_DEBUG_MODE: 
        print("----- SHAPES ------\n")
        print(f"Train labels shape: {train_labels.shape}")
        print(f"Train images shape: {train_images.shape}")

        print(f"Verify labels shape: {verify_labels.shape}")
        print(f"Verify images shape: {verify_images.shape}\n\n")

    model.fit(
        [train_images[:, 0], train_images[:, 1]], train_labels, 
        epochs=EPOCS, 
        validation_data=([verify_images[:, 0], verify_images[:, 1]], verify_labels), verbose=1)
    
    return model

train_data = get_data("Train")
verify_data = get_data("Verify")
trained_model = fit_model(base_model, train_data, verify_data)

Epoch 1/10
22/22 [==============================] - 49s 2s/step - loss: 1.3797 - accuracy: 0.3066 - val_loss: 1.3569 - val_accuracy: 0.3338
Epoch 2/10
22/22 [==============================] - 52s 2s/step - loss: 1.3394 - accuracy: 0.3452 - val_loss: 1.3213 - val_accuracy: 0.3450
Epoch 3/10
22/22 [==============================] - 53s 2s/step - loss: 1.3058 - accuracy: 0.3727 - val_loss: 1.2972 - val_accuracy: 0.3837
Epoch 4/10
22/22 [==============================] - 53s 2s/step - loss: 1.2692 - accuracy: 0.4027 - val_loss: 1.2700 - val_accuracy: 0.3913
Epoch 5/10
22/22 [==============================] - 54s 2s/step - loss: 1.2569 - accuracy: 0.4075 - val_loss: 1.2565 - val_accuracy: 0.4112
Epoch 6/10
22/22 [==============================] - 54s 2s/step - loss: 1.2370 - accuracy: 0.4225 - val_loss: 1.2356 - val_accuracy: 0.4100
Epoch 7/10
22/22 [==============================] - 54s 2s/step - loss: 1.2109 - accuracy: 0.4507 - val_loss: 1.2221 - val_accuracy: 0.4269
Epoch 8/10
22/22 [==

In [14]:
# Save trained model 
trained_model.save(os.path.join(MODEL_PATH, MODEL_NAME))

In [15]:
# Load model
model = keras.models.load_model(os.path.join(MODEL_PATH, MODEL_NAME))


In [17]:
# Test model
test_data = get_data("Test")
test_labels = np.array([map_labels_to_nummeric(label) for label in test_data[1]])

if IN_DEBUG_MODE:
    model.summary()

test_images = normalize_images(np.array((test_data[0])))
predictions = model.predict([test_images[:, 0], test_images[:, 1]])

label_index = random.randint(0, len(test_labels)-1)

print("\n\n")
print(f"------ PREDICTION: Index {label_index + 1} --------\n")
predicted_nummeric = np.argmax(predictions, axis=-1)
predicted_readable = np.vectorize(label_mapping.get)(predicted_nummeric)
actual_readable = np.vectorize(label_mapping.get)(test_labels)


print("NUMMERIC: \n")
print(predicted_nummeric[label_index])
print("READABLE: \n")
print(predicted_readable[label_index])
print("\n\n")

print(f"------ ACTUAL: Index {label_index + 1} ------ \n")
print("NUMMERIC: \n")
print(test_labels[label_index])
print("READABLE: \n")
print(actual_readable[label_index])

4/4 [==============================] - 5s 1s/step



------ PREDICTION: Index 6 --------

NUMMERIC: 

[2 1 0 1 3 1 2 0]
READABLE: 

['blue' 'yellow' 'red' 'yellow' '' 'yellow' 'blue' 'red']



------ ACTUAL: Index 6 ------ 

NUMMERIC: 

[3 1 1 2 3 0 2 0]
READABLE: 

['' 'yellow' 'yellow' 'blue' '' 'red' 'blue' 'red']
